Mount GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Setup working directory

In [ ]:
import os
# set base dir
BASE_DIR='main_proj_v1' 
os.environ['BASE_DIR'] = 'main_proj_v1'
!mkdir -p $BASE_DIR
os.chdir(BASE_DIR)
# get git repo
!git init .
!git remote add -t * -f origin https://github.com/rdj0im/rl-atsc.git
!git checkout main
# set up credentials
!sed -i '/\[remote \"origin\"\]/Q' .git/config
!cat /content/gdrive/MyDrive/Colab\ Notebooks/config >> .git/config

Install SUMO

In [ ]:
!sudo add-apt-repository -y ppa:sumo/stable
!sudo apt-get update
!sudo apt-get -y install sumo sumo-tools sumo-doc


Set sumo env vars

In [ ]:
# Set environment variable
os.environ['SUMO_HOME'] = '/usr/share/sumo'
os.environ['LIBSUMO_AS_TRACI'] = '1' #Optional: for a huge performance boost (~8x) with Libsumo (No GUI)

**SUMO RL**

Install modules

In [ ]:
# !pip install 'setuptools<67'
# !pip install gym==0.21.0
# !pip install stable_baselines3
!pip install git+https://github.com/DLR-RM/stable-baselines3.git
# !pip install sumo-rl
!pip install git+https://github.com/LucasAlegre/sumo-rl.git

Main



In [ ]:
import os
import sys
import sumo_rl
import gym
from stable_baselines3.dqn.dqn import DQN


if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")
import traci

from sumo_rl import SumoEnvironment


if __name__ == "__main__":
    env = sumo_rl.environment.env.SumoEnvironment(
        net_file="sumo_files/4x4_grid_network.net.xml",
        route_file="sumo_files/4x4_grid_routes.rou.xml",
        out_csv_name="dqn",
        single_agent=True,
        use_gui=False,
        num_seconds=100000,
    )

    model = DQN(
        env=env,
        policy="MlpPolicy",
        learning_rate=0.001,
        learning_starts=0,
        train_freq=1,
        target_update_interval=500,
        exploration_initial_eps=0.05,
        exploration_final_eps=0.01,
        verbose=0,
    )
    model.learn(total_timesteps=100000)
    model.save('/content/gdrive/dqn_2way_sing')


---

**Save, Commit and Push**

In [ ]:
# import time
# from IPython.display import display, Javascript
# import hashlib

# # trigger save of current notebook 

# def save_notebook(file_path):
#     start_md5 = hashlib.md5(open(file_path,'rb').read()).hexdigest()
#     display(Javascript('IPython.notebook.save_checkpoint();'))
#     current_md5 = start_md5

#     while start_md5 == current_md5:
#         time.sleep(1)
#         current_md5 = hashlib.md5(open(file_path,'rb').read()).hexdigest()

# copy saved nb to current dir
# file_path='/content/gdrive/MyDrive/Colab Notebooks/main_proj_v1.ipynb'
# save_notebook(file_path)

# 🥲
# ''' Trigger a Save using Ctrl+S beforehand '''
# 🥲

!cp /content/gdrive/MyDrive/Colab\ Notebooks/main_proj_v1.ipynb main_proj_v1.ipynb 
# push to remote
!git add -A
!git commit -m "ipynb save triggered from nb"
!git push
